# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [1]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


NameError: name 'file_path_list' is not defined

In [2]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [3]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [4]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS tumelo
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """
    )

except Exception as e:
    print(e)

#### Set Keyspace


In [5]:
try:
    session.set_keyspace('tumelo')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:  
Give me the artist, song title and song's length in the music app history that was heard during 
sessionId = 338, and itemInSession = 4     

Selecting the artist name, song name, and song length from **song_sessions** table and filtering by sessionId and itemInSession will give the required data for the query. CQL statement:
_SELECT artist, song, length FROM song_sessions WHERE sessionId = 338 AND itemInSession = 4_

Table **song_sessions** will consist of these columns: sessionId, itemInSession, artist, song and length.

The primary key will consist of partition key: **sessionId**, clustering key is **itemInSession** which will enable filtering.

In [6]:
session.execute("""
    CREATE TABLE IF NOT EXISTS song_sessions
    (sessionId int, itemInSession int, artist text, song text, length float,
    PRIMARY KEY(sessionId, itemInSession))
    """)

In [7]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
        query = "INSERT INTO song_sessions (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), (artist), (song), float(length)))

NameError: name 'csv' is not defined

In [ ]:
rows = session.execute('''
                SELECT artist, song, length FROM song_sessions WHERE sessionId = 338 AND itemInSession = 4
                ''')

for row in rows:
    print(row.artist, row.song, row.length)

## Query 2:

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Selecting the artist name, song name, user name and user lastname from the **user_sessions** table and filtering by userId and sessionId will give the required data for the query then order by itemInSession. CQL statement:
_SELECT itemInSession, artist, song, firstName, lastName FROM user_sessions WHERE userId = 10 AND sessionId = 182_

Table **user_sessions** will consist of these columns: userId, sessionId, artist, song, itemInSession, firstName, lastName

The table will consist of the composite partition key **userId** and **sessionId**. The only key used as a partition key is userId, the sessionid which belongs to the same user that will be put into different nodes as part of the partition tolerances and the clustering key is be itemInSession to enable ordering.       

In [8]:
session.execute("""
                CREATE TABLE IF NOT EXISTS user_sessions (userId int, sessionId int, artist text, song text, itemInSession int, firstName text, lastName text, 
                PRIMARY KEY((userId, sessionId), itemInSession))
                """)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        
        query = "INSERT INTO user_sessions (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s,%s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), (artist), song, int(itemInSession), firstName, lastName))

NameError: name 'csv' is not defined

In [ ]:
rows = session.execute("""
                        SELECT itemInSession, artist, song, firstName, lastName FROM user_sessions WHERE userId = 10 AND sessionId = 182
                        """)
for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)


## Query 3:

Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Selecting the user first name and lastname from **music_app_history** table will give the required data for the query then filtering on song name. Due to the user name and lastname not being unique, we will add the column userId to uniquely identify users. . CQL statement: _SELECT firstName, lastName FROM music_app_history WHERE song = 'All Hands Against His Own'_

The table **music_app_history** will consist of these columns: song, firstName, lastName and userId.

The primary key will consist of a partition key song the clustering key will be userId, thus uniquely identifying the queried rows.

In [ ]:
session.execute("""
    CREATE TABLE IF NOT EXISTS music_app_history
    (song text, firstName text, lastName text, userId int,
    PRIMARY KEY(song, userId))
    """)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_app_history (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, firstName, lastName, int (userId)))

In [ ]:
rows = session.execute("""SELECT firstName, lastName FROM music_app_history WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

#### We will drop the tables before closing out the sessions¶

In [ ]:
session.execute(""" DROP TABLE music_app_history """)
session.execute(""" DROP TABLE user_sessions """)
session.execute(""" DROP TABLE song_sessions """)

In [ ]:
#### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()